In [ ]:
import numpy as np
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = [6, 4]
plt.rcParams['figure.dpi'] = 60

In [ ]:
df_pre = pd.read_csv('data/sf_clean.csv', sep=',')
df = df_pre.loc[(df_pre['patient'] < 31)]
adrenal = df.loc[(df['target']=='adrenal_glands')]
boost = df.loc[(df['target']=='boost')]
pancreas = df.loc[(df['target']=='pancreas')]
# df.columns

In [ ]:
sns.scatterplot(data=adrenal, x='distance', y='sparing_factor', hue='patient', palette='Set1')

In [ ]:
g = sns.FacetGrid(adrenal, hue="patient", col="oar")
g.map(sns.scatterplot, 'distance', 'sparing_factor', alpha=.7)
g.add_legend()

In [ ]:
g = sns.FacetGrid(adrenal, hue="patient", row='main_oar', col="oar")
g.map(sns.scatterplot, 'distance', 'sparing_factor', alpha=.7)
g.add_legend()

In [ ]:
adrenal_main = adrenal.loc[adrenal['limiting']==1]
fig, ax1 = plt.subplots()

ax1 = sns.scatterplot(data=adrenal_main, x='patient', y='sparing_factor', hue='oar', palette='Set2')

In [ ]:
fig, ax1 = plt.subplots()

adrenal_oar = adrenal_main.loc[(adrenal_main['oar']=='duodenum')]

sns.scatterplot(data=adrenal_oar, x='fraction', y='sparing_factor', hue='patient', palette='Set1', ax=ax1)
ax2 = ax1.twinx()
ax2 = sns.scatterplot(data=adrenal_oar, x='fraction', y='distance', hue='patient', palette='Set1', marker='x')
ax2.invert_yaxis()

In [ ]:
sns.relplot(data=adrenal_main, x='fraction', y='sparing_factor', hue='patient', palette='Set1', kind='line')

# adrenal_main.plot(x='fraction', y='sparing_factor', kind='line', label='patient')

# for name, group in adrenal_main.groupby("patient"):
#     plt.plot(group.fraction, group.sparing_factor, marker='.', linewidth=2, alpha=0.9, label=name)
# plt.legend()

# sns.lmplot(data=adrenal_main, x="distance", y="sparing_factor", hue="patient")

In [ ]:
g = sns.FacetGrid(df.loc[df['target']!='adrenal_glands'], hue="patient", col="target")
g.map(sns.scatterplot, 'distance', 'sparing_factor', alpha=.7)
g.add_legend()

In [ ]:
sns.scatterplot(data=df.loc[(df['target']=='pancreas')], x='distance', y='sparing_factor', hue='patient', palette='Set1')

In [ ]:
# sns.histplot(df_filtered['sparing_factor'], bins=21)

In [ ]:
# frac = df_filtered['fraction'].values
# emap = colors.ListedColormap(['red', 'green'])
# sns.scatterplot('distance', 'sparing_factor',
#                 hue='oar', data=frac, palette='Set1')
# frac_scat = plt.scatter(df_filtered['distance'], df_filtered['sparing_factor'], c=frac)